In [1]:
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

img_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = MNIST(root='./data/MNIST', download=False, train=True, transform=img_transform)

to_pil_image[0][0]

ModuleNotFoundError: No module named 'torchvision'